# Feature engineering notebook

### •  The aim of this notebook is to apply feature enginnering and prepare data for the next step

### •  This includes to find new features with domain knowledge feature then apply date and time features then string operations to avoid the redunant strings keeping our data clean

## import libraries that is going to be used in our project 

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import time
import random
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.figure_factory as ff
from iso639 import languages
import user_agents
from ip2geotools.databases.noncommercial import DbIpCity as ip2geo
import requests
import json
from geopy.distance import great_circle
from geopy.geocoders import Nominatim
from datasist.structdata import detect_outliers
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OneHotEncoder
from category_encoders import BinaryEncoder 
from datetime import datetime
from sklearn.model_selection import train_test_split , cross_validate
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE , RandomOverSampler
from imblearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.metrics import r2_score , mean_squared_error , classification_report ,recall_score , precision_score , ConfusionMatrixDisplay , confusion_matrix
from sklearn.linear_model import Ridge , Lasso
from sklearn.tree import DecisionTreeRegressor , DecisionTreeClassifier
from sklearn.tree import plot_tree
from sklearn.ensemble import RandomForestRegressor , RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB , GaussianNB , BernoulliNB
from sklearn.svm import SVC , SVR
from sklearn.ensemble import AdaBoostClassifier , GradientBoostingClassifier
import xgboost as xgb
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings('ignore')
import pickle
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
import scipy.cluster.hierarchy as sch
from sklearn.cluster import AgglomerativeClustering,KMeans,DBSCAN
from sklearn.decomposition import PCA
%matplotlib inline
from imblearn.base import _ParamsValidationMixin

## import the cleaned version of data 

In [2]:
raw_data=pd.read_csv('../data/processed/clean_df.csv')
df=raw_data.copy()
df.sample(5)

,Unnamed: 0,age,annual_income,monthly_inhand_salary,interest_rate,num_of_loan,delay_from_due_date,num_of_delayed_payment,num_credit_inquiries,credit_mix,outstanding_debt,credit_utilization_ratio,credit_history_age,payment_of_min_amount,total_emi_per_month,amount_invested_monthly,payment_behaviour,monthly_balance,credit_score
41996,41996,27,33818.63,0.000000,1,4_,20,0,2.0,Good,672.08,33.801359,0,No,68.082204,128.80985766291315,Low_spent_Large_value_payments,326.6298553128432,Good
85319,85319,20,75010.52,6167.876667,21,7,48,19,11.0,Bad,4094.73,23.842316,10 Years and 11 Months,Yes,401.861687,126.46622497981241,High_spent_Large_value_payments,328.45975463919893,Poor
80646,80646,25,9413.305,1072.442083,19,-100,28,17_,11.0,Standard,1746.62,34.158025,18 Years and 5 Months,Yes,32.742096,93.04277361908849,Low_spent_Small_value_payments,271.4593383002017,Poor
5320,5320,48,83483.96,6952.996667,11,1,11,0,1.0,Good,152.98,24.529245,30 Years and 4 Months,No,42.742361,267.012769651052,Low_spent_Small_value_payments,675.5445362371895,Standard
46176,46176,49,72099.14,6079.261667,2,4,15,2,4.0,Good,1421.95,27.044605,27 Years and 7 Months,No,189.886104,591.3193487371998,Low_spent_Medium_value_payments,106.72071365835951,Good


## drop unnecessary csv column

In [3]:
df.drop('Unnamed: 0',axis=1,inplace=True)

## domain knowledge feature

In [4]:
#new two columns to convert years and months into year column and month column in the credit_history_age columns
#year column
def convert_years(x):
    return int(x.split('Years')[0])  
df['credit_history_year']=df['credit_history_age'].apply(convert_years)
df

,age,annual_income,monthly_inhand_salary,interest_rate,num_of_loan,delay_from_due_date,num_of_delayed_payment,num_credit_inquiries,credit_mix,outstanding_debt,credit_utilization_ratio,credit_history_age,payment_of_min_amount,total_emi_per_month,amount_invested_monthly,payment_behaviour,monthly_balance,credit_score,credit_history_year
0,23,19114.12,1824.843333,3,4,3,7,4.0,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good,22
1,23,19114.12,0.000000,3,4,-1,0,4.0,Good,809.98,31.944960,0,No,49.574949,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good,0
2,-500,19114.12,0.000000,3,4,3,7,4.0,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good,22
3,23,19114.12,0.000000,3,4,5,4,4.0,Good,809.98,31.377862,22 Years and 4 Months,No,49.574949,199.4580743910713,Low_spent_Small_value_payments,223.45130972736786,Good,22
4,23,19114.12,1824.843333,3,4,6,0,4.0,Good,809.98,24.797347,22 Years and 5 Months,No,49.574949,41.420153086217326,High_spent_Medium_value_payments,341.48923103222177,Good,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,25,39628.99,3359.415833,7,2,23,7,3.0,_,502.38,34.663572,31 Years and 6 Months,No,35.104023,60.97133255718485,High_spent_Large_value_payments,479.866228,Poor,31
99996,25,39628.99,3359.415833,7,2,18,7,3.0,_,502.38,40.565631,31 Years and 7 Months,No,35.104023,54.18595028760385,High_spent_Medium_value_payments,496.65161,Poor,31
99997,25,39628.99,3359.415833,5729,2,27,6,3.0,Good,502.38,41.255522,31 Years and 8 Months,No,35.104023,24.02847744864441,High_spent_Large_value_payments,516.809083,Poor,31
99998,25,39628.99,3359.415833,7,2,20,0,3.0,Good,502.38,33.638208,31 Years and 9 Months,No,35.104023,251.67258219721603,Low_spent_Large_value_payments,319.164979,Standard,31


In [5]:
#month column
def convert_months(x):
    return x[-9:-7]
df['credit_history_month']=df['credit_history_age'].apply(convert_months)
df

,age,annual_income,monthly_inhand_salary,interest_rate,num_of_loan,delay_from_due_date,num_of_delayed_payment,num_credit_inquiries,credit_mix,outstanding_debt,credit_utilization_ratio,credit_history_age,payment_of_min_amount,total_emi_per_month,amount_invested_monthly,payment_behaviour,monthly_balance,credit_score,credit_history_year,credit_history_month
0,23,19114.12,1824.843333,3,4,3,7,4.0,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good,22,1
1,23,19114.12,0.000000,3,4,-1,0,4.0,Good,809.98,31.944960,0,No,49.574949,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good,0,
2,-500,19114.12,0.000000,3,4,3,7,4.0,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good,22,3
3,23,19114.12,0.000000,3,4,5,4,4.0,Good,809.98,31.377862,22 Years and 4 Months,No,49.574949,199.4580743910713,Low_spent_Small_value_payments,223.45130972736786,Good,22,4
4,23,19114.12,1824.843333,3,4,6,0,4.0,Good,809.98,24.797347,22 Years and 5 Months,No,49.574949,41.420153086217326,High_spent_Medium_value_payments,341.48923103222177,Good,22,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,25,39628.99,3359.415833,7,2,23,7,3.0,_,502.38,34.663572,31 Years and 6 Months,No,35.104023,60.97133255718485,High_spent_Large_value_payments,479.866228,Poor,31,6
99996,25,39628.99,3359.415833,7,2,18,7,3.0,_,502.38,40.565631,31 Years and 7 Months,No,35.104023,54.18595028760385,High_spent_Medium_value_payments,496.65161,Poor,31,7
99997,25,39628.99,3359.415833,5729,2,27,6,3.0,Good,502.38,41.255522,31 Years and 8 Months,No,35.104023,24.02847744864441,High_spent_Large_value_payments,516.809083,Poor,31,8
99998,25,39628.99,3359.415833,7,2,20,0,3.0,Good,502.38,33.638208,31 Years and 9 Months,No,35.104023,251.67258219721603,Low_spent_Large_value_payments,319.164979,Standard,31,9


In [6]:
#check na values for the two new columns
df[['credit_history_year','credit_history_month']].isna().sum()

credit_history_year     0
credit_history_month    0
dtype: int64

In [7]:
#check special variables for the two new columns
lst_domain_knowledge=['credit_history_year','credit_history_month']
for i in lst_domain_knowledge :
    print(f'for feature {i} : \n',df[i].value_counts())
    print('-'*30)

for feature credit_history_year : 
 credit_history_year
0     9520
19    4784
18    4747
17    4718
16    4526
15    3854
20    3325
6     3054
13    3048
30    2982
32    2899
8     2814
29    2812
9     2803
31    2799
12    2790
23    2789
28    2767
24    2764
26    2727
22    2721
21    2708
11    2703
27    2622
10    2618
7     2575
25    2522
14    2360
5     2033
33    1106
1     1001
2      938
3      809
4      762
Name: count, dtype: int64
------------------------------
for feature credit_history_month : 
 credit_history_month
      9030
11    8046
10    8026
 5    7972
 4    7950
 3    7925
 8    7901
 2    7894
 9    7884
 0    6920
 1    6885
 6    6829
 7    6738
Name: count, dtype: int64
------------------------------


In [8]:
#for month new columns we have found there is blank value which represnets no months or equivelant to zero
df['credit_history_month']=df['credit_history_month'].replace('',0)

In [9]:
#convert month column into numerical value
df['credit_history_month']=df['credit_history_month'].astype(int)

In [10]:
#check special variables again after revamping 
df['credit_history_month'].value_counts()

credit_history_month
0     15950
11     8046
10     8026
5      7972
4      7950
3      7925
8      7901
2      7894
9      7884
1      6885
6      6829
7      6738
Name: count, dtype: int64

In [11]:
#drop credit_history_age column because it is redunant
df.drop('credit_history_age',axis=1,inplace=True)

## string operations

In [12]:
#lets apply a deep string operations on multiple columns some of them needed to be converted into numerical data type
#take over view on categorical columns and see what we can do with them
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 19 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   age                       100000 non-null  object 
 1   annual_income             100000 non-null  object 
 2   monthly_inhand_salary     100000 non-null  float64
 3   interest_rate             100000 non-null  int64  
 4   num_of_loan               100000 non-null  object 
 5   delay_from_due_date       100000 non-null  int64  
 6   num_of_delayed_payment    100000 non-null  object 
 7   num_credit_inquiries      100000 non-null  float64
 8   credit_mix                100000 non-null  object 
 9   outstanding_debt          100000 non-null  object 
 10  credit_utilization_ratio  100000 non-null  float64
 11  payment_of_min_amount     100000 non-null  object 
 12  total_emi_per_month       100000 non-null  float64
 13  amount_invested_monthly   100000 non-null  ob

In [13]:
#age column
df['age'].value_counts()

age
38      2833
28      2829
31      2806
26      2792
32      2749
        ... 
471        1
1520       1
8663       1
3363       1
1342       1
Name: count, Length: 1788, dtype: int64

In [14]:
#here we found age columns consists of 4 digits some times which is not logical so we can take the first two digits 
def age(x):
    return x[:2]
df['age']=df['age'].apply(age)

In [15]:
df['age']

0        23
1        23
2        -5
3        23
4        23
         ..
99995    25
99996    25
99997    25
99998    25
99999    25
Name: age, Length: 100000, dtype: object

In [16]:
#we also has the '_' symbol to be replaced with space better
df['age']=df['age'].replace('_',' ')
df['age'].value_counts()

age
38    3018
28    2990
31    2973
26    2968
32    2904
      ... 
90       1
93       1
91       1
87       1
97       1
Name: count, Length: 91, dtype: int64

In [17]:
#convert age column to positive int
def edit_age(x):
    return abs(int(x))
df['age']=df['age'].apply(edit_age)

In [18]:
#check the other columns special variables that needed to be edited
lst_num=['annual_income', 'num_of_loan', 'num_of_delayed_payment','outstanding_debt','amount_invested_monthly','monthly_balance']
for i in lst_num :
    print(f'for feature {i} : \n',df[i].value_counts())
    print('-'*60)

for feature annual_income : 
 annual_income
36585.12     16
20867.67     16
17273.83     16
9141.63      15
33029.66     15
             ..
20269.93_     1
15157.25_     1
44955.64_     1
76650.12_     1
4262933.0     1
Name: count, Length: 18940, dtype: int64
------------------------------------------------------------
for feature num_of_loan : 
 num_of_loan
3        14386
2        14250
4        14016
0        10380
1        10083
         ...  
1320_        1
103          1
1444         1
392          1
966          1
Name: count, Length: 434, dtype: int64
------------------------------------------------------------
for feature num_of_delayed_payment : 
 num_of_delayed_payment
0       8570
19      5327
17      5261
16      5173
10      5153
        ... 
848_       1
4134       1
1530       1
1502       1
2047       1
Name: count, Length: 749, dtype: int64
------------------------------------------------------------
for feature outstanding_debt : 
 outstanding_debt
1360.45     24
460

In [19]:
#we have found that the rest of numerical columns  has the '_' symbol to be replaced with space better (convert it to string first) then reconvert it to numerical value (int)
def edit_rest(x):
    x=str(x)
    x=x.replace('_',' ')
    return int(x.split('.')[0])
for i in lst_num:
    df[i]=df[i].apply(edit_rest)

In [20]:
#convert the monthly_balanced column to numerical variable
df['monthly_balance']=df['monthly_balance'].astype(float)
df['monthly_balance']=df['monthly_balance'].astype(int)

In [21]:
#check general inforamtion of edited columns
df[lst_num].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 6 columns):
 #   Column                   Non-Null Count   Dtype
---  ------                   --------------   -----
 0   annual_income            100000 non-null  int64
 1   num_of_loan              100000 non-null  int64
 2   num_of_delayed_payment   100000 non-null  int64
 3   outstanding_debt         100000 non-null  int64
 4   amount_invested_monthly  100000 non-null  int64
 5   monthly_balance          100000 non-null  int32
dtypes: int32(1), int64(5)
memory usage: 4.2 MB


## extract clean version of df to be used in our next phases with encoding= 'latin-1' to avoid any kind of errors 

In [28]:
df.to_csv('../data/processed/featured_df.csv',encoding = 'latin-1')